# data loading

In [1]:

import argparse
# from audioop import rms
# import logging
import os
# from re import L
import sys
# from itertools import product
from time import localtime, sleep, strftime, time

import json
import numpy as np
import setproctitle # to set the name of process
import torch
import torch.utils
import torch.nn as nn
# from tensorboardX import SummaryWriter
# from torch import multiprocessing as mp # 多线程工作
# from dataset import get_data_queue_cf, get_data_queue_cf_nonsparse
from dataset import get_data_queue_efficiently, get_data_queue_negsampling_efficiently # original graph
# from dataset import get_data_queue_subsampling_efficiently, get_data_queue_subsampling_efficiently_explicit # subsample
from models import (GMF, MLP,  NGCF)
# from controller import sample_arch_cf, sample_arch_cf_signal, sample_arch_cf_test
from train_eval import (evaluate_cf_efficiently_implicit, train_single_cf_efficiently )
%load_ext autoreload
%autoreload 2

/home/abc/anaconda3/envs/dlrs/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser(description="Run.")
parser.add_argument('--lr', type=float, default=0.05, help='init learning rate')
# parser.add_argument('--arch_lr', type=float, default=0.05, help='learning rate for arch encoding')
parser.add_argument('--controller_lr', type=float, default=1e-1, help='learning rate for controller')
parser.add_argument('--weight_decay', type=float, default=1e-5, help='weight decay')
# parser.add_argument('--update_freq', type=int, default=1, help='frequency of updating architeture')
parser.add_argument('--opt', type=str, default='Adagrad', help='choice of opt')
parser.add_argument('--use_gpu', type=int, default=1, help='whether use gpu')
parser.add_argument('--minibatch', type=int, default=1, help='whether use minibatch')
parser.add_argument('--gpu', type=int, default=0, help='gpu device id')
parser.add_argument('--train_epochs', type=int, default=2000, help='num of training epochs')
parser.add_argument('--search_epochs', type=int, default=1000, help='num of searching epochs')
parser.add_argument('--save', type=str, default='save/', help='experiment name')
parser.add_argument('--seed', type=int, default=1, help='random seed')
parser.add_argument('--grad_clip', type=float, default=5, help='gradient clipping')
parser.add_argument('--train_portion', type=float, default=0.5, help='portion of training data')
parser.add_argument('--valid_portion', type=float, default=0.25, help='portion of validation data')
parser.add_argument('--dataset', type=str, default='ml-100k', help='dataset')
parser.add_argument('--mode', type=str, default='random_single', help='search or single mode')
parser.add_argument('--process_name', type=str, default='AutoCF@wenyan', help='process name')
parser.add_argument('--embedding_dim', type=int, default=2, help='dimension of embedding')
# parser.add_argument('--controller', type=str, default='PURE', help='structure of controller')
# parser.add_argument('--controller_batch_size', type=int, default=4, help='batch size for updating controller')
# parser.add_argument('--unrolled', action='store_true', default=True, help='use one-step unrolled validation loss')
parser.add_argument('--max_batch', type=int, default=65536, help='max batch during training')
parser.add_argument('--device', type=int, default=0, help='GPU device')
parser.add_argument('--multi', type=int, default=0, help='using multi-training for single architecture')
parser.add_argument('--if_valid', type=int, default=1, help='use validation set for tuning single architecture or not')
parser.add_argument('--breakpoint', type=str, default='save/log.txt', help='the log file storing existing results')
parser.add_argument('--arch_file', type=str, default='src/arch.txt', help='all arches')
parser.add_argument('--remaining_arches', type=str, default='src/arch.txt', help='')
parser.add_argument('--arch_assign', type=str, default='[0,3]', help='')
parser.add_argument('--data_type', type=str, default='implicit', help='explicit or implicit(default)')
parser.add_argument('--loss_func', type=str, default='bprloss', help='Implicit loss function')
parser.add_argument('--mark', type=str, default='') # 
parser.add_argument('--batch_size', type=int, default=5000, help='batch size')
# parser.add_argument('--file_id', type=int, default=100, help='file id')

# parser.add_argument('--use_sample', type=int, default=0, help='whether use data subgraph')
# parser.add_argument('--sample_portion', type=float, default=0.20, help='portion of data subgraph')
# parser.add_argument('--sample_mode', type=str, default='distribute', help='topk or distribute mode of sampling')

args = parser.parse_args([])

In [3]:
dim = 2
args.dataset = 'ml-100k'
data_path = args.dataset + '/'
# setting datasets,  default='ml-100k'
if args.dataset == 'ml-100k': # default
    num_users = 943
    num_items = 1682
args.num_users = num_users
args.num_items = num_items
# only implicit data
train_queue_pair, valid_queue, test_queue = get_data_queue_negsampling_efficiently(data_path, args)
train_queue_pair = [k.cuda() for k in train_queue_pair]
train_queue_pair[3] = train_queue_pair[3].to_sparse()
train_queue_pair[4] = train_queue_pair[4].to_sparse() # 
print(train_queue_pair[4])

# data for evaluation
all_users = torch.tensor(list(range(num_users)), dtype=torch.int64).repeat_interleave(num_items)
all_items = torch.tensor(list(range(num_items)), dtype=torch.int64).repeat(num_users)
with torch.cuda.device(args.device):
    all_users = all_users.cuda()
    all_items = all_items.cuda()

100000
943 1682
tensor(indices=tensor([[   0,    0,    0,  ..., 1676, 1677, 1680],
                       [   0,    1,    4,  ...,  853,  862,  895]]),
       values=tensor([5., 4., 4.,  ..., 3., 1., 3.]),
       device='cuda:0', size=(1682, 943), nnz=50000, layout=torch.sparse_coo)


# norm ngcf

In [4]:
R = train_queue_pair[3].to_dense()
R.shape # num_users = 943, num_items = 1682
zero_num_users = torch.zeros(num_users, num_users, device=args.device)
zero_num_items = torch.zeros(num_items, num_items, device=args.device)
adj_mat = torch.concat([torch.concat([zero_num_users, R], dim=1),
                torch.concat([R.T, zero_num_items], dim=1)], dim=0) # adjacency matrix
# A.shape
user_freqency = torch.sum(R > 0.0, axis=1) # train
# degree_user_array = # freqency
item_freqency = torch.sum(R > 0.0, axis=0)
# user_freqency.shape, item_freqency.shape
freq_tensor = torch.concat([user_freqency, item_freqency], dim=0)
degree_diag = torch.diag(freq_tensor) # degree
# D.shape 
degree_diag_calc = torch.diag(freq_tensor**(-1/2)) # D^(-1/2)
# torch.sum(degree_diag_calc == torch.inf) 94
# degree_diag_calc[torch.where(degree_diag_calc == torch.inf)] == 0.0
degree_diag_calc = torch.where(torch.isinf(degree_diag_calc), torch.full_like(degree_diag_calc, 0), degree_diag_calc)

laplace_mat = degree_diag_calc @ adj_mat @ degree_diag_calc # L
# laplace_mat[:20, :20]
# freq_tensor.shape[0], np.count_nonzero(freq_tensor.cpu()), freq_tensor.shape[0] - np.count_nonzero(freq_tensor.cpu())

In [10]:
args.embedding_dim = 3
# should be changed to encoding
embedding_ui = torch.randn(args.num_users + args.num_items, args.embedding_dim,device=args.device)# random input 
eye_ui = torch.eye(num_items + num_users, device=args.device)
dl_0 = args.embedding_dim # encoding
dl_1 = 6 # first embedding when l =1
dl_2 = 10
initializer = nn.init.xavier_uniform_
_W1_1 = nn.Linear(dl_0, dl_1,  bias=False, device=args.device)
_W2_1 = nn.Linear(dl_0, dl_1,  bias=False, device=args.device)
_W1_2 = nn.Linear(dl_1, dl_2,  bias=False, device=args.device)
_W2_2 = nn.Linear(dl_1, dl_2,  bias=False, device=args.device)
# _W1 = nn.Parameter(initializer(torch.empty(dl_1, dl_2)))
# _W2 = nn.Linear(num_items + num_users, dl_1, bias=False, device=args.device)

sigmoid_func = nn.LeakyReLU(negative_slope=0.2)
# inference = sigmoid_func(_W1((laplace_mat + eye_ui) @ embedding_ui)
            #   (laplace_mat@embedding_ui)*_W2(embedding_ui))
embedding_ui_0 = embedding_ui # (N+M)*embedding
embedding_ui_1 = sigmoid_func(_W1_1((laplace_mat + eye_ui) @ embedding_ui_0) + _W2_1((laplace_mat @ embedding_ui_0)*embedding_ui_0))
embedding_ui_2 = sigmoid_func(_W1_2((laplace_mat + eye_ui) @ embedding_ui_1) + _W2_2((laplace_mat @ embedding_ui_1)*embedding_ui_1)) # three layers
# enbedding
embedding_cat = torch.cat([embedding_ui_0, embedding_ui_1, embedding_ui_2], dim=1)



tensor([[-0.1131, -0.4657,  0.4829,  ..., -0.2209, -0.1275,  0.5211],
        [ 0.1473,  0.5953,  1.5309,  ..., -0.2834, -0.2603,  0.0750],
        [ 0.1485, -0.3977, -0.4761,  ..., -0.1025, -0.0662,  0.1304],
        ...,
        [ 0.2413,  0.3738, -0.8026,  ..., -0.0268,  0.0077,  0.0985],
        [-0.2896, -1.5091, -1.3163,  ...,  0.1187,  0.1617, -0.0040],
        [-0.7377, -0.8497,  0.4920,  ...,  0.0733,  0.0357, -0.0063]],
       device='cuda:0', grad_fn=<CatBackward0>)

In [6]:
# embedding_ui_1, embedding_ui_1.shape, embedding_ui_2, embedding_ui_2.shape

In [8]:
_UsersEmbedding = nn.Embedding(num_users, args.embedding_dim, device=args.device)
print(users_train.shape, _UsersEmbedding(users_train).shape)


torch.Size([50000]) torch.Size([50000, 3])


In [9]:
model = NGCF(num_users, num_items, args.embedding_dim, args.weight_decay)

model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

losses = []
performances = []
start = time()
for train_epoch in range(args.train_epochs):
    loss = train_single_cf_efficiently(train_queue_pair, model, optimizer, args)
    
    losses.append(loss)
    if dim == 2: # default 2， 验证集合上的操作
        if 'DMF' in args.mode or 'JNCF' in args.mode: 
            rmse = evaluate_cf_efficiently_implicit(model, test_queue, all_users, all_items, args)
        elif args.mode == 'NGCF':
            rmse = evaluate_cf_efficiently_implicit(model, test_queue, all_users, all_items, args)
        else: # GMF MLP 
            rmse  = evaluate_cf_efficiently_implicit(model, test_queue, all_users, all_items, args) 
    performances.append(rmse)
    if train_epoch % 100 == 0 or train_epoch == args.train_epochs - 1:
        print('train_epoch: %d, loss: %.4f, recall20: %.4f[%.4f]' % (train_epoch, loss, rmse, time()-start))


TypeError: __init__() missing 1 required positional argument: 'norm_adj'

In [ ]:
import numpy as np
a = np.array([1,2,3])
b = np.array([3,2,1])
a*b

array([3, 4, 3])

In [4]:
import torch
import torch.nn as nn

In [5]:
nn.Parameter(torch.Tensor(3,2))

Parameter containing:
tensor([[ 1.6489e-16,  4.5643e-41],
        [-2.9683e-16,  3.0653e-41],
        [ 4.4842e-44,  0.0000e+00]], requires_grad=True)